In [1]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from sklearn.linear_model import LinearRegression
import numpy as np
from regression_model import RegressionModel
from sklearn.preprocessing import PolynomialFeatures

In [2]:
df = pd.read_csv('data/Gold.csv')
df.tail()

,Unnamed: 0,Daily_Return,EMA10,EMA100,EMA100Cross,EMA10Cross,EMA12,EMA12Cross,EMA20,EMA20Cross,...,SMA15Cross,SMA20,SMA20Cross,SMA5,SMA50,SMA50Cross,SMA5Cross,Settle_Price,Time,Volume
10972,12/4/2019,-0.003046,1318.427611,1308.988215,0,0,1318.710813,0,1319.847607,0,...,0,1321.430,0,1320.62,1327.696,0,0,1313.4,11073,2352
10973,15/4/2019,-0.010845,1316.786228,1308.996369,0,0,1317.278380,0,1318.852597,0,...,0,1321.050,0,1320.56,1327.086,0,0,1309.4,11074,2647
10974,16/4/2019,-0.000386,1312.861459,1308.723174,1,0,1313.881706,0,1316.599969,0,...,0,1320.535,0,1318.46,1326.456,0,0,1295.2,11075,2819
10975,17/4/2019,-0.000463,1309.559375,1308.445487,0,0,1310.930675,0,1314.514257,0,...,0,1319.050,0,1312.24,1325.596,0,0,1294.7,11076,1677
10976,18/4/2019,0.000000,1306.748580,1308.161418,0,0,1308.341340,0,1312.570042,0,...,0,1317.795,0,1304.80,1324.724,0,0,1294.1,11077,1130


In [3]:
columns = list(df.columns)
numerics = [x for x in columns[2:-3] if 'Cross' not in x and x != 'MACD']

In [4]:
for col in numerics:
    df[col+'norm'] = df[col] / df['Settle_Price'] - 1

In [5]:
columns = list(df.columns)
features = [col for col in columns if 'norm' in col]

In [6]:
X = df[features]
Y = df['Daily_Return']

In [ ]:
trainsplit = int(len(Y) * 0.9)
validsplit = trainsplit #+ int(len(Y) * 0.1)
Xtrain = X[:trainsplit]
Ytrain = Y[:trainsplit]
Xvalid = X[trainsplit:validsplit]
Yvalid = Y[trainsplit:validsplit]
Xtest = X[validsplit:]
Ytest = Y[validsplit:]

In [ ]:
model = RegressionModel(5)
print model.getModelInfo()
model.train(Xtrain, Ytrain)

Interaction and Polynomial


/Users/divya/.local/lib/python2.7/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [ ]:
predicted_returns = model.predict(Xtest)
plt.plot(Ytest, predicted_returns, 'k.')
plt.title("Prediction accuracy")
plt.xlabel("True returns")
plt.ylabel("Predicted returns")

In [ ]:
strongest_signal = np.max(predicted_returns)
Ytest = Ytest.values

In [ ]:
# Long Only Portfolio
portfolio_weight = 1.0
portfolio_over_time = [1.0]
returns = []
max_bet_pct = 1.0 # Percent of holdings to risk on the best bet
for t in range(len(predicted_returns)):
    r_pred = predicted_returns[t]
    r_true = Ytest[t]
    if r_pred < 0:
        portfolio_over_time.append(portfolio_weight)
        returns.append(0.0)
        continue
        
    bet_size_pct = max_bet_pct * r_pred / strongest_signal
    bet_size_portfolio = portfolio_weight * bet_size_pct
    portfolio_weight = portfolio_weight + bet_size_portfolio * r_true
    portfolio_over_time.append(portfolio_weight)
    returns.append(r_true * bet_size_portfolio / portfolio_weight)

In [ ]:
portfolio_weight

In [ ]:
plt.plot(portfolio_over_time)
plt.xlabel("Timestep")
plt.ylabel("Current:Original Holdings Ratio")

In [ ]:
plt.hist(returns, bins=30)

In [ ]:
output = 1.0
for y in Ytest:
    output *= (1.0 + y)
output